数据/图画要求 
+ a 双个脚压力相同
+ b 左脚压力最大，右脚压力0（过程）
+ c 左脚压力减小，右脚压力增加（过程）
+ d 左脚压力0，右脚压力最大（过程）
+ e 右脚压力减小，左脚压力增大；(过程)
<br>
按照：abcdebcde......这样进行循环；
<br>
过程c和e可以分为3个不同比例体现动态；



1. pressure_data：双脚压力相同
2. interpolated_data:左脚压力最大，右脚压力0

In [18]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from cop_trajectory_analysis.data_loader import PressureDataLoader, CoordinateDataLoader
from cop_trajectory_analysis.cop_calculation import (
    left_map_rule,
    right_map_rule,
    calculate_distances,
    calculate_F_COP,
    calculate_cop_xy,
)

def process_pressure_data():
    """
    start_end_data:                存有各个阶段数据的起始和结束数据
    left_right_equal:              两个脚相同压力数据也就是最开始的数据
    left_max_right_zero:           左脚压力最大、右脚压力0(过程)
    left_decrease_right_increase:  左脚压力减小，右脚压力增加（过程）
    left_zero_right_max:           左脚压力0，右脚压力最大（过程）
    right_decrease_left_increase:  右脚压力减小，左脚压力增大（过程）
    
        
    """ 
    df=pd.read_csv('data/start_end_data.csv')
    df=df.drop(index=[0,3,6,9,12])
    df = df.drop(df.columns[-1], axis=1)
    df = df.reset_index(drop=True)
    data = df.to_numpy()
    data = data.astype(float) 
    # 将每两行转换为一个2x16的矩阵
    n = data.shape[0] // 2
    result = np.array([data[i * 2 : (i + 1) * 2, :] for i in range(n)])
    data=[]
    for i in range(n):
        if(i+1 == n): break
        time_points = np.array([0, 1])  # t=0 对应开始数据, t=1 对应左脚支撑数据
        positions = np.array([result[i], result[i+1]])  # 对应的重心位置
        # 创建样条插值模型
        cs = CubicSpline(time_points, positions)
        # 在0到1之间生成插值点
        t_values = np.linspace(0, 1, 25)
        interpolated_data = cs(t_values)
        data.append(interpolated_data)
    return data

In [19]:
all_period_data=process_pressure_data()
reshaped_data = [element.reshape(-1, 17) for element in all_period_data]

# 初始化 timestamp 计数器
timestamp_counter = 0

for i in range(len(reshaped_data)):
    df = pd.DataFrame(reshaped_data[i], columns=[f"sensor{i}" for i in range(17)])
    df.columns.values[0] = "foot"
    df["timestamp"] = [timestamp_counter+i // 2 for i in range(len(df))]
    timestamp_counter += len(df) // 2
    df.to_csv(f"data/{i}.csv", index=False)

In [20]:
from scipy.interpolate import CubicSpline

# 定义数据点（时间和对应的重心位置）

def create_rotated_cop_xy():
    """
    0文件-[11,11]--->[6,11]:          从两脚相同到左脚最大，右脚为0
    1文件-[6,11]--->[11,11]:          从左脚压力最大到右脚为0 到 左脚压力减小，右脚压力增加
    2文件-[11,11]-->[16,11]:          从左脚压力减小，右脚压力增大 到 左脚压力为0，右脚压力最大
    3文件-[16,11]-->[11,11]:          从左脚压力减小，右脚压力增加 到 右脚压力减小，左脚压力增大

    """

    time_points = np.array([0,1,2,3,4])  # t=0 对应开始数据, t=1 对应左脚支撑数据
    positions = np.array([[11,11],[6,11],[11,11],[16,11],[11,11]])  # 对应的重心位置

    # 创建样条插值模型
    cs = CubicSpline(time_points, positions)

    # 在0到1之间生成插值点
    t_values = np.linspace(0, 4, 125)
    interpolated_data = cs(t_values)
    return interpolated_data

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import cv2
import matplotlib.image as mpimg


def create_heatmap(
    index, total_sensor_xy, total_pressure, rotated_cop_xy_list, out_video
):
    # 自定义颜色映射
    colors = [
        (1.0, 1.0, 1.0, 0.0),  # 白色
        (0.0, 1.0, 1.0, 1.0),  # 青色
        (0.0, 0.0, 1.0, 1.0),  # 蓝色
        (0.0, 1.0, 0.0, 1.0),  # 绿色
        (1.0, 1.0, 0.0, 1.0),  # 黄色
        (1.0, 0.0, 0.0, 1.0),  # 红色
    ]
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
    figure_data = np.zeros(shape=(22, 22))
    for (x, y), pressure in zip(total_sensor_xy, total_pressure):
        figure_data[x, y] = pressure

    smoothed_data = gaussian_filter(figure_data, sigma=0.7)
    max_smoothed = np.max(smoothed_data)
    norma_smoothed = smoothed_data / max_smoothed

    rotated_norma_smoothed = np.rot90(norma_smoothed)

    # 创建平滑热力图
    plt.figure(figsize=(10, 8))
    plt.imshow(
        rotated_norma_smoothed,
        cmap=cmap,
        interpolation="bilinear",
        vmin=0,
        vmax=0.99,
        zorder=6,
        alpha=0.5,
    )
    # 读取图片
    left = mpimg.imread("assets/left-1.png")
    right = mpimg.imread("assets/right-1.png")
    # 显示图片并指定显示区域
    plt.imshow(
        left, extent=[2, 7, 15, 6], zorder=5
    )  # 图片显示在 (2, 8) 和 (3, 7) 的区域
    plt.imshow(
        right, extent=[15, 20, 15, 6], zorder=5
    )  # 图片2显示在 (6, 9) 和 (2, 5) 的区域
    plt.colorbar(label="%")
    plt.title("平滑热力图")
    plt.xlabel("X 轴")
    plt.ylabel("Y 轴")

    tail_length = 50
    for i, point in enumerate(rotated_cop_xy_list):
        if i < index:
            # 当前点之后的点，形成拖尾效果
            alpha = min((index - i) / tail_length, 1)  # 透明度
            plt.scatter(
                *point,
                edgecolor=(1 - alpha, 0, alpha),
                facecolor="none",
                s=5,
                zorder=5,
                alpha=alpha
            )  # 其他点为空心
        elif i == index:
            plt.scatter(*point, color="red", s=250, zorder=5)  # 突出显示当前点
        else:
            plt.scatter(
                *point, edgecolor="black", facecolor="none", s=5, zorder=5, alpha=0.2
            )  # 其他点为空心

    plt.plot(
        rotated_cop_xy_list[:, 0],
        rotated_cop_xy_list[:, 1],
        color="grey",
        linestyle="--",
    )  # 连接轨迹

    # plt.scatter(rotated_cop_xy_list[index][0],rotated_cop_xy_list[index][1], color="black", s=100, zorder=5)
    # 创建点状背景
    x, y = np.meshgrid(np.arange(figure_data.shape[1]), np.arange(figure_data.shape[0]))
    plt.scatter(x, y, color="black", s=10, alpha=0.5, zorder=5)
    # 保存当前帧
    plt.savefig("temp_frame.png")
    plt.close()

    # 读取帧并写入视频
    img = cv2.imread("temp_frame.png")
    out_video.write(img)

In [22]:
def create_image_data(index, file_paths):
    pressure_loader = PressureDataLoader(file_path=file_paths)
    pressure_data = pressure_loader.load_pressure()[index]
    coordinate_loader = CoordinateDataLoader(
        file_path="data/coordinate_data.csv",
        left_map=left_map_rule,
        right_map=right_map_rule,
    )
    coordinate_data = coordinate_loader.load_coordinates()["2025-02-11T15:50:51"]
    total_sensor_xy = coordinate_loader.get_total_coordinate()["2025-02-11T15:50:51"]
    total_pressure = pressure_loader.get_total_pressure()[index]
    return total_sensor_xy, total_pressure

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
import cv2

file_paths=['data/0.csv','data/1.csv','data/2.csv','data/3.csv']    

rotated_cop_xy_list = create_rotated_cop_xy()

# rotated_cop_xy_list
# 视频设置
size = (1000, 800)  # 根据实际图片大小设置
out = cv2.VideoWriter("output_video.avi", cv2.VideoWriter_fourcc(*"XVID"), 10, size)
for t in range(100):
    totoal_sensor_xy, total_pressure = create_image_data(t,file_paths)
    create_heatmap(
        index=t,
        total_sensor_xy=totoal_sensor_xy,
        total_pressure=total_pressure,
        rotated_cop_xy_list=rotated_cop_xy_list,
        out_video=out,
    )
out.release()

/var/folders/cb/8_fz48_s1c14r0z10lvg3hb00000gn/T/ipykernel_17273/1588769268.py:90: UserWarning: Glyph 36724 (\N{CJK UNIFIED IDEOGRAPH-8F74}) missing from current font.
  plt.savefig("temp_frame.png")
/var/folders/cb/8_fz48_s1c14r0z10lvg3hb00000gn/T/ipykernel_17273/1588769268.py:90: UserWarning: Glyph 24179 (\N{CJK UNIFIED IDEOGRAPH-5E73}) missing from current font.
  plt.savefig("temp_frame.png")
/var/folders/cb/8_fz48_s1c14r0z10lvg3hb00000gn/T/ipykernel_17273/1588769268.py:90: UserWarning: Glyph 28369 (\N{CJK UNIFIED IDEOGRAPH-6ED1}) missing from current font.
  plt.savefig("temp_frame.png")
/var/folders/cb/8_fz48_s1c14r0z10lvg3hb00000gn/T/ipykernel_17273/1588769268.py:90: UserWarning: Glyph 28909 (\N{CJK UNIFIED IDEOGRAPH-70ED}) missing from current font.
  plt.savefig("temp_frame.png")
/var/folders/cb/8_fz48_s1c14r0z10lvg3hb00000gn/T/ipykernel_17273/1588769268.py:90: UserWarning: Glyph 21147 (\N{CJK UNIFIED IDEOGRAPH-529B}) missing from current font.
  plt.savefig("temp_frame.png")


轨迹不能重合，到左边的时候，要到左边脚的中心。到右边的时候，到右边脚的中心